In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train =scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [11]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
    
])

model.compile(loss='mean_squared_error', optimizer='sgd')
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7842 - val_loss: 0.7693
Epoch 2/20
363/363 [==============================] - 0s 911us/step - loss: 3.6416 - val_loss: 0.5264
Epoch 3/20
363/363 [==============================] - 0s 875us/step - loss: 0.5312 - val_loss: 0.4046
Epoch 4/20
363/363 [==============================] - 0s 853us/step - loss: 0.4051 - val_loss: 0.3923
Epoch 5/20
363/363 [==============================] - 0s 877us/step - loss: 0.4033 - val_loss: 0.6584
Epoch 6/20
363/363 [==============================] - 0s 996us/step - loss: 0.3857 - val_loss: 0.5624
Epoch 7/20
363/363 [==============================] - 0s 926us/step - loss: 0.3727 - val_loss: 0.3429
Epoch 8/20
363/363 [==============================] - 0s 902us/step - loss: 0.3601 - val_loss: 0.3398
Epoch 9/20
363/363 [==============================] - 0s 904us/step - loss: 0.3542 - val_loss: 0.3480
Epoch 10/20
363/363 [==============================] - 0s 939us/step - loss: 0.3473 

In [5]:
y_test[:3]

array([1.5  , 1.024, 3.413])

In [4]:
y_pred

array([[1.621834 ],
       [2.0484762],
       [2.8577476]], dtype=float32)

### 함수형 API 복잡한 모델 만들기
   - Wide & Deep 신경망 만들기
   - 입력의 일부 또는 전체가 출력층에 바로연결
   - 신경망을 복잡한 패턴과 간단한 규칙을 모두 학습가능

In [7]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])


### 입력이 여러개 인경우
 - 입력을 다중으로 입력하고 싶은경우가 있음. 
 - ex) 상품정보를 predict하는 모델을 만들경우 흔히 상품페이지에는 상품명과 하위클래스로 옵션명이 있다 
 -     이러한 경우 상품명과 옵션명을 동시에 입력으로 넣어 predict 하도록 설계하였다


In [7]:
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name='output')(concat)
model = keras.Model(inputs= [input_A, input_B], outputs=[output])


In [11]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                   validation_data=((X_valid_A, X_valid_B), y_valid))

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))


Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 2.1226 - val_loss: 0.9642
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.8555 - val_loss: 0.7331
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7158 - val_loss: 0.6659
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6673 - val_loss: 0.6322
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6388 - val_loss: 0.6072
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6151 - val_loss: 0.5880
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5962 - val_loss: 0.5683
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5796 - val_loss: 0.5548
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5651 - val_loss: 0.5396
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5526 - val_loss: 0.5286

### 여러 출력이 필요한 경우
- 여러 출력이 필요한 작업일 경우 
- ex) 그림에있는 주요 물체를 분류하고 위치를 알아야 할 수 있습니다. 이때, 회귀 작업 과 분류 작업을 함께 하는 경우 입니다(Google Vision Api 가 제공하는 작업)


In [12]:
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name='main_output')(concat)
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [15]:
# 각 출력층은 각각의 loss fuction이 필요하기 때문에 리스트로 전달
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1], optimizer='sgd')


In [16]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.9922 - main_output_loss: 0.8820 - aux_output_loss: 1.9837 - val_loss: 0.6449 - val_main_output_loss: 0.5744 - val_aux_output_loss: 1.2797
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7380 - main_output_loss: 0.6849 - aux_output_loss: 1.2158 - val_loss: 0.5860 - val_main_output_loss: 0.5277 - val_aux_output_loss: 1.1108
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5803 - main_output_loss: 0.5302 - aux_output_loss: 1.0311 - val_loss: 0.5273 - val_main_output_loss: 0.4813 - val_aux_output_loss: 0.9415
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5299 - main_output_loss: 0.4892 - aux_output_loss: 0.8958 - val_loss: 0.5066 - val_main_output_loss: 0.4682 - val_aux_output_loss: 0.8530
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5130 - main_output_loss: 0.4778 - aux_output_loss: 0.8299 - val_loss

In [17]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test,y_test])

162/162 [==============================] - 0s 1ms/step - loss: 0.3718 - main_output_loss: 0.3576 - aux_output_loss: 0.4999


In [18]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

In [25]:
y_pred_main

array([[1.6289976],
       [2.1785896],
       [2.965984 ]], dtype=float32)

In [23]:
y_pred_aux

array([[1.380136 ],
       [2.2462208],
       [3.3289845]], dtype=float32)

### 서브클래싱 API로 동적 모델 만들기

In [5]:
class WideAndDeepModel(keras.Model):# keras의 Model 클래스를 상속한 다음 생성자 안에 필요한 층 생성
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
    
model = WideAndDeepModel()
        
    

## 모델 저장 복원

In [ ]:
# 케라스의 HDF5 포맷을 사용하여 모델 구조와 층의 모든 모델 파라미터를 저장합니다.

In [ ]:
model = keras.models.Sequential([...])
model.complie([...])
model.fit([...])
model.save(';;;.h5')


In [ ]:
model = keras.models.load_model(';;;.h5')


### 콜백 사용하기

In [12]:
checkpoint_db = keras.callbacks.ModelCheckpoint('my_keras_model.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint_db])

Epoch 1/10
363/363 [==============================] - 0s 741us/step - loss: 0.3291
Epoch 2/10
363/363 [==============================] - 0s 697us/step - loss: 0.3399
Epoch 3/10
363/363 [==============================] - 0s 761us/step - loss: 0.3332
Epoch 4/10
363/363 [==============================] - 0s 687us/step - loss: 0.3273
Epoch 5/10
363/363 [==============================] - 0s 769us/step - loss: 0.3249
Epoch 6/10
363/363 [==============================] - 0s 714us/step - loss: 0.3242
Epoch 7/10
363/363 [==============================] - 0s 745us/step - loss: 0.3239
Epoch 8/10
363/363 [==============================] - 0s 744us/step - loss: 0.3248
Epoch 9/10
363/363 [==============================] - 0s 675us/step - loss: 0.3220
Epoch 10/10
363/363 [==============================] - 0s 736us/step - loss: 0.3215


In [13]:
### 최상의 모델로 복원
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras_model.h5', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid),
                   callbacks=[checkpoint_cb])
model = keras.models.load_model('my_keras_model.h5')

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.3215 - val_loss: 0.3179
Epoch 2/10
363/363 [==============================] - 0s 971us/step - loss: 0.3220 - val_loss: 0.3138
Epoch 3/10
363/363 [==============================] - 0s 944us/step - loss: 0.3205 - val_loss: 0.3146
Epoch 4/10
363/363 [==============================] - 0s 888us/step - loss: 0.3194 - val_loss: 0.3199
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.3183 - val_loss: 0.3113
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.3176 - val_loss: 0.3025
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.3181 - val_loss: 0.3143
Epoch 8/10
363/363 [==============================] - 0s 930us/step - loss: 0.3163 - val_loss: 0.3137
Epoch 9/10
363/363 [==============================] - 0s 949us/step - loss: 0.3173 - val_loss: 0.3087
Epoch 10/10
363/363 [==============================] - 0s 908us/step - loss: 0.3150 - val_

In [14]:
# early_stopping 사용하기
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
histroy = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                   callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3170 - val_loss: 0.3287
Epoch 2/100
363/363 [==============================] - 0s 941us/step - loss: 0.3160 - val_loss: 0.3085
Epoch 3/100
363/363 [==============================] - 0s 963us/step - loss: 0.3160 - val_loss: 0.3117
Epoch 4/100
363/363 [==============================] - 0s 919us/step - loss: 0.3194 - val_loss: 0.3196
Epoch 5/100
363/363 [==============================] - 0s 904us/step - loss: 0.3163 - val_loss: 0.3073
Epoch 6/100
363/363 [==============================] - 0s 964us/step - loss: 0.3148 - val_loss: 0.3051
Epoch 7/100
363/363 [==============================] - 0s 968us/step - loss: 0.3158 - val_loss: 0.3123
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3142 - val_loss: 0.3006
Epoch 9/100
363/363 [==============================] - 0s 941us/step - loss: 0.3131 - val_loss: 0.3076
Epoch 10/100
363/363 [==============================] - 0s 980us/step - loss:

In [ ]:
### 

#연습문제

- 역전파란 무엇이고 어떻게 작동하나요? 역전파와 후진 모드 자동 미분의 차이점은 무엇인가요?

-> 먼저 모델의 모든 파리미터(가중치와 편향)대한 비용 함수의 그래디언트를 계산하고, 이 그래디언트를 사용해 경사 하강법 스텝을 수행합니다. 역전파 단계는 모델 파라미터가 비용함수를 최소화하는 값으로 수렴할 때까지 훈련 배치에서 일반적으로 수천 혹은 수백만 번 수행됩니다.
그래디언트를 계산하기 위해 연전파는 후진 모드 자동 미분을 사용합니다. 후진 모드 자동 미분은 계산 그래프의 정방향 계산에서 현재 훈련 배치에 대한 모든 노드의 값을 구합니다. 그 다음에 역방향 계산에서 한번에 모든 그래디어트를 구합니다. 역전파는 그래디언트 계산과 경사 하강법 스텝을 여러 번 수행하여 인공 신경망을 훈련시키는 전체 프로세스를 의미합니다. 이와 다르게 후진 모드 자동 미분은 그래디언트를 효과적으로 계산하는 하나의 기법으로 역전파에서 사용됩니다.
